# Sensor Simulator

In the previous notebook
[`bayesian_optimization_blooper.ipynb`](bayesian_optimization_blooper.ipynb), we ran
into some unexpected results. Bayesian optimization was on par with grid search, and
random search was the best? See below.

![blooper](bayesian_optimization_blooper.png)

To help troubleshoot the source of the unexpected
behavior, we'll move the schedule up and use a simulator. The large stochasticity for
identical inputs shown at the end of the last notebook is a cause for concern.

|  | mean | std |
|---|---|---|
| ch415_violet | 17928.500000 | 25288.711738 |
| ch445_indigo | 8746.200000 | 9379.586248 |
| ch480_blue | 19396.100000 | 17832.023101 |
| ch515_cyan | 4870.900000 | 12126.813252 |
| ch560_green | 11908.300000 | 18944.797128 |
| ch615_yellow | 18406.400000 | 19626.981147 |
| ch670_orange | 30584.100000 | 20868.707815 |
| ch720_red | 5776.100000 | 9066.354228 |
| ch_clear | 1294.100000 | 2017.116669 |
| ch_nir | 0.000000 | 0.000000 |
| mae | 13798.390000 | 2959.743242 |
| rmse | 23851.888103 | 3422.810056 |

Is the stochasticity due to excessive noise from too short of integration time or is it
a problem with the hardware such as a sensor malfunction? Based on inspection from my
own eyes, fixed inputs appeared to produce very similar colors. Last, this could be due
to (intentional) naive design decision of starting out with mean absolute error for the
objective function rather than something more tuned to this problem, such as Wasserstein
distance between the two discrete spectra. In reality, it's probably some mix and
interplay of the previous issues:

- **(epistemic) uncertainty**
  - i.e. just a part of the system, solved by a longer integration time (more data)
- **sensor malfunction/degradation**
  - due to blasting it repeatedly with a bright DotStar
  - and/or hard device resets due to system crashes
  - and/or or (just remembered!) a series
  of recent unexpected power outages for my housing community
- **Poorly defined objective function**
  - It was defined in a way that it doesn't handle stochastic bias when
  a signal is measured in an adjacent channel, solved by using a robust distance metric
  for discrete distributions (e.g. Wasserstein)
- **Something else?**

My wife kindly reminded me of past lessons learned during experimentation, and
encouraged me to try out the simulation. So, let's do that here! We'll take a look at
our domain knowledge / assumptions for the simulation, briefly describe the approach,
and then dig into a snapshot of the `SensorSimulator` class. Finally, we'll demonstrate
the usage and run our grid vs. random vs. Bayesian search algorithm comparison with our
simulation.


## Domain Knowledge

> TLDR; extract the data from the DotStar RGB spectrum listed in the [manufacturer's datasheet](https://cdn-shop.adafruit.com/product-files/2343/SK9822_SHIJI.pdf)

A basic physics course teaches the wave-particle duality of light. Light can be thought
of as a wave (signal with frequency/wavelength) or as a particle (photon). In the case
of designing a (very basic) simulation for an RGB LED spectrum being read by a
discrete-channel sensor, we focus mainly on the wave portion of wave-particle duality of
light. Most light-emitting-diodes (LEDs) have fairly narrow distributions of wavelengths
that they emit like what's shown in the following image from the [DotStar manufacturer's datasheet](https://cdn-shop.adafruit.com/product-files/2343/SK9822_SHIJI.pdf).

<img src="../reports/figures/dotstar/rgb-relative-emission-vs-wavelength.png" width=350>

Controlling the brightness and RGB values of our LED controls the relative contribution
of each of the three distributions portrayed above. Meanwhile, our spectrophotometer (a device that measures the intensity of light at various
wavelengths) measures the light intensity at 8 wavelengths (it also has "clear" and
"near infrared" channels that we'll ignore):

| wavelength | intensity |
|---|---|
415 nm | Violet
445 nm | Indigo or Blue
480 nm | Blue or Blue-Green
515 nm | Blue-Green or Green
555 nm | Green or Yellow-Green 
590 nm | Yellow-Green or Yellow
630 nm | Orange or Orange-Red or Red
680 nm | Red

We can digitize the data from the manufacturer datasheet using
[WebPlotDigitizer](https://automeris.io/WebPlotDigitizer/) and then use that for
calculating/mixing an arbitrary spectrum based on brightness and RGB settings. We'll make the
reasonable approximation/assumption that the [superposition principle](https://en.wikipedia.org/wiki/Superposition_principle) applies*. In other
words, we assume that the individual spectra from the red, green, and blue LEDs can be
added together. Below is a screenshot of the datapoints extracted via the
WebPlotDigitizer interface for each of the three curves. Corresponding `.csv` files are
also saved: [`red.csv`](../src/self_driving_lab_demo/data/red.csv), [`green.csv`](../src/self_driving_lab_demo/data/green.csv), and [`blue.csv`](../src/self_driving_lab_demo/data/blue.csv).

<img src="../src/self_driving_lab_demo/data/wpd-rgb-spectrum-points-overlay.png"
width=350>

See [RGB LEDs vs. having 10+ monochromatic light
sources](https://github.com/sparks-baird/self-driving-lab-demo/issues/6) for more
details on the hardware design considerations for the LEDs and sensor.

<p><sup>
*While it would be interesting to look at optimization setups that
involve light cancellation, the equipment required is likely not within the budget
and time constraints of the self-driving-lab-demo project (less than 100 USD and less
than an hour of setup time).
</p></sup>


## Simulation Details

As mentioned, we'll mix the three spectra extracted from the manufacturer datasheet
according to the simulation inputs (brightness
and RGB values). After loading the CSV data, we'll clip any data below 0.0 and average
the intensities that are not one-to-one (i.e. multiple intensities for the same
wavelength) due to the data extraction process. Next, we'll use `scipy`'s `interp1d`
function for each of the three color spectra to create linear interpolation functions
which are set to zero outside of the range of the original data (extrapolation).
Finally, we'll calculate a weighted sum of each of the interpolators as a function of
brightness and RGB sampled at each of the wavelengths mentioned above.

A more sophisticated setup might involve sampling a distribution of wavelengths for each
channel; however, we'll keep it simple for now.

## Sensor Simulator Python class
Below is a snapshot of the `SensorSimulator` class, broken into chunks.

First, we need our imports and define the wavelengths we'll be sampling at as a constant.

```python
from importlib.resources import open_text
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from self_driving_lab_demo import data as data_module

CHANNEL_WAVELENGTHS = [
    415,
    445,
    480,
    515,
    560,
    615,
    670,
    720,
]
```

The class doesn't take any keyword arguments (again, keeping it simple), and when the
class is instantiated, it creates an interpolator.

```python
class SensorSimulator(object):
    def __init__(self):
        self.red_interp = self.create_interpolator("red.csv")
        self.green_interp = self.create_interpolator("green.csv")
        self.blue_interp = self.create_interpolator("blue.csv")

```

We make it easy to get the channel wavelengths (making it a constant outside the class
makes it easily accessible for other modules).

```python
    @property
    def channel_wavelengths(self):
        return CHANNEL_WAVELENGTHS
```

The data is read using best practices (`open_text` using Python modules), negative
values are clipped, and y-values with repeat x-values are averaged. Finally, the
`interp1d` uses a linear interpolation (`cubic` gave some outlandish values during
testing) and zero-ing extrapolation (zero anywhere outside the range of the original
dataset). This is done separately for each of the red, green, and blue curves.

```python
    def create_interpolator(self, fname):
        df = pd.read_csv(
            open_text(data_module, fname),
            header=None,
            names=["wavelength", "relative_intensity"],
        )

        df["relative_intensity"].clip(lower=0.0, inplace=True)

        # average y-values for repeat x-values
        # see also https://stackoverflow.com/a/51258988/13697228
        df = df.groupby("wavelength", as_index=False).mean()

        return interp1d(
            df["wavelength"],
            df["relative_intensity"],
            kind="linear",
            bounds_error=False,
            fill_value=0.0,
        )

```

To perform the weighted average (mixing) of spectra, we divide the RGB values by 255
(this is arbitrary) and multiply by the brightness. This is then multipled by the
interpolated value at each of the wavelengths. Finally, we sum the contribution of each
of the wavelengths.

```python
    def _simulate_sensor_data(self, wavelengths, brightness, R, G, B):
        rI, gI, bI = brightness * np.array([R, G, B]) / 255
        channel_data = np.sum(
            [
                self.red_interp(wavelengths) * rI,
                self.green_interp(wavelengths) * gI,
                self.blue_interp(wavelengths) * bI,
            ],
            axis=0,
        )
        return tuple(channel_data)
```

Last, we define a class method that fixes the wavelengths to the constant mentioned before.

```python
    def simulate_sensor_data(self, brightness, R, G, B):
        return self._simulate_sensor_data(self.channel_wavelengths, brightness, R, G, B)
```

## Algorithm Comparison